In [13]:
import subprocess
import time
import picamera
import numpy as np

MIN_F = 70
MAX_F = 7000
MIN_I = 0
MAX_I = 1785

# There is only this one sound process
proc = sound_proc(440)

def avg_color(rgb_array):
    return np.mean(rgb_array, axis=(0, 1))

def color_to_int(c):
 The maximum should be 4095 (12-bit)    return 256*round(c[0]/16 - 0.5) + 16*round(c[1]/16 - 0.5) + round(c[2]/16 - 0.5)

def int_to_freq(i):
    return round(MAX_F - ((MAX_F - MIN_F) / (MAX_I - MIN_I))*i)

# Make a new process every time we need a new frequency
def sound_proc(freq):
    return subprocess.Popen(['speaker-test', '--frequency', str(freq), '--test', 'sine'])

if __name__ == "__main__":
    try:
        global proc
        img_output = np.empty((240, 320, 3), dtype=np.uint8)  # This is where we dump the image

        with picamera.PiCamera() as camera:
            camera.resolution = (320, 240)
            time.sleep(2)
            while True:
                camera.capture(img_output, 'rgb')
                proc.kill()  # Kill the old frequency
                freq = int_to_freq(color_to_int(avg_color(img_output)))
                proc = sound_proc(freq)
                time.sleep(0.1)
    except Exception as e:
        print(e)
    finally:
        proc.kill()

KeyboardInterrupt: 